In [2]:
from py2neo import Graph, Subgraph
from py2neo import Node, Relationship, Path
from py2neo import NodeMatcher

# 连接数据库
graph = Graph('bolt://localhost:7687', auth=('neo4j', '123456'))
nodes = NodeMatcher(graph)

In [7]:
# 删除所有已有节点
# graph.delete_all()

In [2]:
import mysql.connector

# 连接数据库
connection = mysql.connector.connect(
    host='localhost',  # 数据库主机地址
    user='root',  # 数据库用户名
    password='123456',  # 数据库密码
    database='relationship'  # 要连接的数据库名称
)
cursor = connection.cursor()

## 导入节点

In [8]:
sql_select_query = "SELECT id, huji_no, address, idcard, name, gender FROM huji"
cursor.execute(sql_select_query)
person_datas = cursor.fetchall()

for p in person_datas:
    # 定义node
    node = Node('Person',id=p[0], huji_no=p[1], address=p[2], idcard=p[3], name =p[4], gender =p[5])
    # 存入图数据库
    graph.create(node)

## 更新节点属性

In [18]:
# 清除原有 community_id 属性
# graph.run("MATCH (n:Person) REMOVE n.community_id")

In [3]:
sql_select_query = "SELECT idcard, community_id FROM community_res where alg_info='leideng_max_comm_15_240702_1'"
cursor.execute(sql_select_query)
community_datas = cursor.fetchall()

for i, (idcard, community_id) in enumerate(community_datas):
    if i%500==0:
        print(f"finish add {i} properties")
    tmp_node = nodes.match("Person", idcard=idcard).first()
    tmp_node['community_id'] = community_id
    graph.push(tmp_node)

# print("finish add 29000 properties")

finish add 29000 properties


## 批量插入关系

64112 -- 106878

In [ ]:
# MATCH ()-[r]-() DELETE r;

In [4]:
sql_select_query = "SELECT src_id, dest_id, avg_time, show_type FROM relationship.relation_statistic_step2"
cursor.execute(sql_select_query)
relation_datas = cursor.fetchall()

transaction = graph.begin()
for i, (src_id, dest_id, rweight,show_type) in enumerate(relation_datas):
    if i%1000==0:
        print(f"finish add {i} relations")
        graph.commit(transaction)
        transaction = graph.begin()
    node_src = nodes.match("Person", idcard=src_id).first()
    node_dest = nodes.match("Person", idcard=dest_id).first()
    rel = Relationship(node_src,show_type,node_dest, weight = rweight)
    transaction.create(rel)
graph.commit(transaction)


# print("finish add 170000 relations")

finish add 170000 relations


## 数据查询

In [1]:
# match(n)-[r]-(m) return n,r,m limit 30
# match(n)-[r]-(m) where n.community_id in ['63','64'] return n
# match(n)-[r*1..2]-(m) where n.idcard='110123193207075050' return n,r,m  # 两层关系
# match()-[r]-() return count(r)

In [ ]:
# ""